In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PyTorch Image Classification Multi-Node Distributed Data Parallel Training on GPU using Vertex Training with Custom Container

 
<table align="left">
<td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pytorch_image_classification_distributed_data_parallel_training_with_vertex_sdk/multi_node_ddp_nccl_vertex_training_with_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
    
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pytorch_image_classification_distributed_data_parallel_training_with_vertex_sdk/multi_node_ddp_nccl_vertex_training_with_custom_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>                                                                                               
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pytorch_image_classification_distributed_data_parallel_training_with_vertex_sdk/multi_node_ddp_nccl_vertex_training_with_custom_container.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview


This tutorial demonstrates how to create a multi-node, distributed image classification using PyTorch on Vertex SDK with GPU. This can help your training job scale to handle large amounts of data.

### Dataset

The dataset used for this tutorial is the <a href="http://yann.lecun.com/exdb/mnist/">MNIST database</a>. The MNIST database of handwritten digits has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.



### Objective

In this tutorial, you learn how to create a distributed PyTorch training job using Vertex AI SDK for Python and custom containers. You will set up GCP to use a custom container, a Vertex Tensorboard Instance and run a custom training job. 

This tutorial uses the following Google Cloud ML services:

- `Vertex AI SDK`
- `Vertex AI TensorBoard`
- `CustomContainerTrainingJob`
- `Artifact Registry`

The steps performed include:

- Setting up your GCP project : Setting up the PROJECT_ID, REGION & SERVICE_ACCOUNT
- Creating a cloud storage bucket
- Building Custom Container using Artifact Registry and Docker
- Create a Vertex AI Tensorboard Instance to store your Vertex AI experiment
- Run a Vertex AI SDK CustomContainerTrainingJob

## Costs
 
This tutorial uses billable components of Google Cloud:

Vertex AI

Cloud Storage

Vertex AI TensorBoard (Note that Vertex AI TensorBoard charges a monthly fee of $300 per unique active user. Active users are measured through the Vertex AI TensorBoard UI. You also pay for Google Cloud resources you use with Vertex AI TensorBoard, such as TensorBoard logs stored in Cloud Storage. <a href='https://cloud.google.com/vertex-ai/pricing#tensorboard'>Check the link for latest prices.</a>)

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/),
        to generate a cost estimate based on your projected usage.


### Install additional packages

Install the latest version of Vertex AI SDK for Python.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.12.0
    Uninstalling google-cloud-aiplatform-1.12.0:
      Successfully uninstalled google-cloud-aiplatform-1.12.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

# Setup

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform

In [3]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [4]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: vertex-ai-dev


In [5]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [31]:
REGION = "[your-region]" # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

#### Service Account
If you don't know your service account, try to get your service account using gcloud command by executing the second cell below.

In [7]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your GCP project id from gcloud
    shell_output = !gcloud auth list 2>/dev/null
    SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()
    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 931647533046-compute@developer.gserviceaccount.com


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [8]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [33]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"

In [34]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [35]:
! gsutil mb -l $REGION $BUCKET_URI

E0426 09:43:04.509806559   14620 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Creating gs://vertex-ai-devaip-20220421133550/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [36]:
! gsutil ls -al $BUCKET_URI

E0426 09:43:07.426595105   14620 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [41]:
content_name = "pt-img-cls-multi-node-ddp-cust-cont"

## Vertex AI Training using a custom container

### Built Custom Container

In [19]:
hostname = "gcr.io"
image_name = content_name
tag = "latest"

custom_container_image_uri = f"{hostname}/{PROJECT_ID}/{image_name}:{tag}"

### Initialize Vertex AI SDK

In [20]:
from google.cloud import aiplatform

aiplatform.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_URI,
    location=REGION,
)

### Create a Vertex AI TensorBoard Instance

NOTE: <a href='https://cloud.google.com/vertex-ai/pricing#tensorboard'>Vertex AI TensorBoard charges a monthly fee of $300 per unique active user. Active users are measured through the Vertex AI TensorBoard UI. You also pay for Google Cloud resources you use with Vertex AI TensorBoard, such as TensorBoard logs stored in Cloud Storage.</a>Please check above link for latest prices.

In [22]:
content_name = content_name + "-gpu"

tensorboard = aiplatform.Tensorboard.create(
    display_name=content_name,
)

Creating Tensorboard
Create Tensorboard backing LRO: projects/931647533046/locations/us-central1/tensorboards/391619653534744576/operations/2560743408356294656
Tensorboard created. Resource name: projects/931647533046/locations/us-central1/tensorboards/391619653534744576
To use this Tensorboard in another session:
tb = aiplatform.Tensorboard('projects/931647533046/locations/us-central1/tensorboards/391619653534744576')


#### Option: Use a previously created Vertex AI TensorBoard instance

```
tensorboard_name = "Your Tensorboard Resource Name or Tensorboard ID"
tensorboard = aiplatform.Tensorboard(tensorboard_name=tensorboard_name)
```

### Run a Vertex AI SDK CustomContainerTrainingJob

In [23]:
display_name = content_name
gcs_output_uri_prefix = f"{BUCKET_URI}/{display_name}"

replica_count = 1
machine_type = "n1-standard-4"
accelerator_count = 4
accelerator_type = "NVIDIA_TESLA_K80"

args = [
    "--backend",
    "nccl",
    "--batch-size",
    "128",
    "--epochs",
    "25",
]

In [24]:
custom_container_training_job = aiplatform.CustomContainerTrainingJob(
    display_name=display_name,
    container_uri=custom_container_image_uri,
)

You can use the Vertex AI Tensorboard `custom_container_training_job` link from the output to view the Accuracy and Loss of model

In [25]:
custom_container_training_job.run(
    args=args,
    base_output_dir=gcs_output_uri_prefix,
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_count=accelerator_count,
    accelerator_type=accelerator_type,
    tensorboard=tensorboard.resource_name,
    service_account=SERVICE_ACCOUNT,
)

Training Output directory:
gs://vertex-ai-devaip-20220421133550/pt-img-cls-multi-node-ddp-cust-cont-gpu 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5946117651326566400?project=931647533046
CustomContainerTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5946117651326566400 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3063198163297894400?project=931647533046
View tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+931647533046+locations+us-central1+tensorboards+391619653534744576+experiments+3063198163297894400
CustomContainerTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5946117651326566400 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/931647533046/locations/us-central1/trainingPipelines/5946117651326566400

In [39]:
print(f"Custom Training Job Name: {custom_container_training_job.resource_name}")
print(f"GCS Output URI Prefix: {gcs_output_uri_prefix}")

Custom Training Job Name: projects/931647533046/locations/us-central1/trainingPipelines/5946117651326566400
GCS Output URI Prefix: gs://vertex-ai-devaip-20220421133550/pt-img-cls-multi-node-ddp-cust-cont-gpu


### View output artifact from training

In [40]:
! gsutil ls $gcs_output_uri_prefix

E0426 09:43:51.240621503   14620 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


CommandException: One or more URLs matched no objects.


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Artifacts
- Cloud Storage Bucket

In [29]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = True

! gsutil rm -rf $gcs_output_uri_prefix

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

E0421 13:56:14.221333541   14620 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


CommandException: 1 files/objects could not be removed.


E0421 13:56:15.982473796   14620 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


Removing gs://vertex-ai-devaip-20220421133550/...
